# Problem: Implement an LSTM Model 

### Problem Statement
You are tasked with implementing a simple **LSTM (Long Short-Term Memory)** model in PyTorch. The model should process sequential data using an LSTM layer followed by a fully connected (FC) layer. Your goal is to complete the model by defining the LSTM layer and implementing the forward pass.

### Requirements
1. **Define the LSTM Model**:
   - Add an **LSTM layer** to the model. This layer should accept input sequences and output hidden states.
   - Add a **fully connected (FC) layer** that maps the output of the LSTM to the final predictions.
   - Implement the `forward` method to:
     - Pass the input sequence through the LSTM.
     - Feed the output of the LSTM into the fully connected layer for the final output.

### Constraints
- The LSTM layer should be implemented with a single hidden layer.
- Use a suitable number of input features, hidden units, and output size for the task.
- Make sure the `forward` method returns the output of the fully connected layer after processing the LSTM output.


<details>
  <summary>💡 Hint</summary>
  Add the LSTM layer and FC layer in LSTMModel.__init__.
  <br>
  Implement the forward pass to process sequences using the LSTM and FC layers.
</details>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
# Generate synthetic sequential data
torch.manual_seed(42)
sequence_length = 10
num_samples = 100

# Create a sine wave dataset
X = torch.linspace(0, 4 * 3.14159, steps=num_samples).unsqueeze(1)
y = torch.sin(X)

# Prepare data for LSTM
def create_in_out_sequences(data, seq_length):
    in_seq = []
    out_seq = []
    for i in range(len(data) - seq_length):
        in_seq.append(data[i:i + seq_length])
        out_seq.append(data[i + seq_length])
    return torch.stack(in_seq), torch.stack(out_seq)

X_seq, y_seq = create_in_out_sequences(y, sequence_length)
X_seq.shape, y_seq.shape

(torch.Size([90, 10, 1]), torch.Size([90, 1]))

In [19]:
# Define the LSTM Model
# TODO: Add LSTM layer, forward implementation
class LSTMModel(nn.Module):
    # 스터디:
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    # 저는 마지막 시퀀스의 hidden state를 활용하지 않고 전체 시퀀스를 기반으로 사용해봤습니다

    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(1, 50, 1) # input_size, hidden_size, num_layers
        self.fc = nn.Linear(50, 1) # upproj 한거 다시 원래대로 돌려놓기
        self.out = nn.Linear(10, 1) # 시퀀스 downproj
    def forward(self, x):
        x, shape = self.lstm(x)
        x = self.fc(x) # 이거 거치면 (90, 10, 1) 이 됨. 각 시퀀스가 90개 차원인듯?
        x = torch.flatten(x, start_dim=1) # 마지막 1 차원 버리기
        x = self.out(x) # (90, 1)로 projection
        return x
# Initialize the model, loss function, and optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Define the LSTM Model
# TODO: Add LSTM layer, forward implementation
class LSTMModel(nn.Module):
    # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
    # 이건 last_hidden_state 활용한 방법

    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(1, 50, 1) # input_size, hidden_size, num_layers
        self.fc = nn.Linear(50, 1) # upproj 한거 다시 원래대로 돌려놓기
    def forward(self, x):
        x, shape = self.lstm(x)
        x = self.fc(x) # 이거 거치면 (90, 10, 1) 이 됨. 각 시퀀스가 90개 차원인듯?
        x = torch.flatten(x, start_dim=1)[:,-1] # 마지막 1 차원을 버리고, 시퀀스의 마지막만 남겨둡니다(90,1)
        return x
# Initialize the model, loss function, and optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [17]:
prediction = model(X_seq)
prediction.shape

torch.Size([90, 10, 50])


torch.Size([90, 1])

In [20]:
# Training loop
epochs = 500
for epoch in range(epochs):
    # Forward pass
    predictions = model(X_seq)
    loss = criterion(predictions, y_seq)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Log progress every 50 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [50/500], Loss: 0.0025
Epoch [100/500], Loss: 0.0005
Epoch [150/500], Loss: 0.0003
Epoch [200/500], Loss: 0.0026
Epoch [250/500], Loss: 0.0002
Epoch [300/500], Loss: 0.0001
Epoch [350/500], Loss: 0.0001
Epoch [400/500], Loss: 0.0001
Epoch [450/500], Loss: 0.0001
Epoch [500/500], Loss: 0.0001


로스 0.001 미만으로 찍히면 통과하겠습니다. RNN이 완전 0을 만들 수 있긴 한데 굳이?

In [21]:
# Testing on new data
test_steps = 20  # Ensure this is greater than sequence_length
X_test = torch.linspace(4 * 3.14159, 5 * 3.14159, steps=test_steps).unsqueeze(1)
y_test = torch.sin(X_test)

# Create test input sequences
X_test_seq, _ = create_in_out_sequences(y_test, sequence_length)

with torch.no_grad():
    predictions = model(X_test_seq)
    print(f"Predictions for new sequence: {predictions.squeeze().tolist()}")


Predictions for new sequence: [0.9700343608856201, 1.0616066455841064, 1.0675443410873413, 1.044981598854065, 1.0027172565460205, 0.9387857913970947, 0.849774956703186, 0.7327950596809387, 0.5865545272827148, 0.4133932590484619]
